In [11]:
import numpy as np
import pandas as pd
import os

In [9]:
df = pd.read_csv("LinkedIn Scraper Results Batch 1.csv")
df = df[df.columns[:-13]]
df = df.drop(columns = ["Email", "Assigned To", "LinkedIn URLs"])
df = df.fillna("0")
df.head()

,Name,Headline,Company,Experiences,Education,Yes/No/Maybe,Notes
0,Andrea Gewirtz,Health & Wealth Strategist at WellBeing & Worth,WellbeingandWorth,"Founder @ WellbeingandWorth, Oct 2019 – Presen...","The Johns Hopkins University, 1986 – 2001\n Pe...",No,0
1,Muskaan N.,Founder at Aromatherapy University,Aromatherapy University,Founder @ Aromatherapy University\n Self-empl...,"Master's degree @ Harvard University, 2019 – 2...",No,0
2,"Michael Morton, CFP®, ChFC®",Financial Advisor for Entrepreneurs and Tech P...,Morton Financial Advice,Founder @ Morton Financial Advice\n Self-empl...,"BA @ Swarthmore College, 1993 – 1997\n MS @ Th...",No,0
3,Alvin Alexander,Founder & CEO at Orion Applied Science & Techn...,"Orion Applied Science & Technology, LLC",Founder & CEO @ Orion Applied Science & Techno...,"MS @ The Johns Hopkins University, 2010 – 2014...",No,0
4,Tracey Miller,"Women's Wellness Exchange, Adventures Within, LLC",Women's Wellness Exchange,"Founder @ Women's Wellness Exchange, Dec 2020 ...",Master’s Degree @ The Johns Hopkins University...,No,0


In [10]:
df.shape

(1311, 7)

In [58]:
education = pd.read_csv("./education/cwurData.csv")
education.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [66]:
education.sort_values(by=['year'], inplace=True, ascending=False)
education.drop_duplicates(subset='institution', keep='first', inplace=True)

In [67]:
print(len(set(schools)))
print(len(schools))

1023
1023


In [68]:
schools = set(education.institution.tolist())

In [82]:
education.loc[education['institution'] == 'University of California, Los Angeles', 'national_rank'].item()

12

In [92]:
def find_university(item): 
    list_of_schoolies = []
    for school in schools: 
        if school in item: 
            list_of_schoolies.append(school)
    return list_of_schoolies

def ranking(schoolies):
    if len(schoolies) == 0: 
        return np.isnan
    else: 
        all_ranks = [education.loc[education['institution'] == i,'national_rank'].item() for i in schoolies]
        avg_rank = sum(all_ranks)/len(schoolies)
        #print(avg_rank)
        return avg_rank
    
def quality(schoolies): 
    if len(schoolies) == 0: 
        return np.isnan
    else: 
        all_ranks = [education.loc[education['institution'] == i,'quality_of_education'].item() for i in schoolies]
        avg_rank = sum(all_ranks)/len(schoolies)
        #print(avg_rank)
        return avg_rank

def founder(exp): 
    f = exp.lower().strip().split().count("founder")
    return f

def self_employed(exp):
    if 'self-employed' in exp.lower(): 
        return True
    else: 
        return False
    
df['schools'] = df['Education'].apply(find_university)
df['national_rank'] = df['schools'].apply(ranking)
df['quality_of_education'] = df.schools.apply(quality)
df['number_founders'] = df.Experiences.apply(founder)
df['self_employed'] = df.Experiences.apply(self_employed)

In [93]:
df.head()

,Name,Headline,Company,Experiences,Education,Yes/No/Maybe,Notes,schools,national_rank,quality_of_education,number_founders,self_employed
0,Andrea Gewirtz,Health & Wealth Strategist at WellBeing & Worth,WellbeingandWorth,"Founder @ WellbeingandWorth, Oct 2019 – Presen...","The Johns Hopkins University, 1986 – 2001\n Pe...",No,0,[Johns Hopkins University],13.0,18.0,2,False
1,Muskaan N.,Founder at Aromatherapy University,Aromatherapy University,Founder @ Aromatherapy University\n Self-empl...,"Master's degree @ Harvard University, 2019 – 2...",No,0,"[University of California, Riverside, Johns Ho...",26.25,68.25,1,True
2,"Michael Morton, CFP®, ChFC®",Financial Advisor for Entrepreneurs and Tech P...,Morton Financial Advice,Founder @ Morton Financial Advice\n Self-empl...,"BA @ Swarthmore College, 1993 – 1997\n MS @ Th...",No,0,[Johns Hopkins University],13.0,18.0,3,True
3,Alvin Alexander,Founder & CEO at Orion Applied Science & Techn...,"Orion Applied Science & Technology, LLC",Founder & CEO @ Orion Applied Science & Techno...,"MS @ The Johns Hopkins University, 2010 – 2014...",No,0,[Johns Hopkins University],13.0,18.0,2,False
4,Tracey Miller,"Women's Wellness Exchange, Adventures Within, LLC",Women's Wellness Exchange,"Founder @ Women's Wellness Exchange, Dec 2020 ...",Master’s Degree @ The Johns Hopkins University...,No,0,[Johns Hopkins University],13.0,18.0,3,True


In [94]:
df["Outcome"] = np.zeros(df.shape[0])

for row in df.index:
    if df["Yes/No/Maybe"][row] == "No":
        df["Outcome"][row] = 0
    if df["Yes/No/Maybe"][row] == "Maybe":
        df["Outcome"][row] = 1
    if df["Yes/No/Maybe"][row] == "Yes":
        df["Outcome"][row] = 2

/var/folders/49/4vmnlx457c1cstpgllx_ck_h0000gn/T/ipykernel_91724/628328432.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Outcome"][row] = 0
/var/folders/49/4vmnlx457c1cstpgllx_ck_h0000gn/T/ipykernel_91724/628328432.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Outcome"][row] = 1
/var/folders/49/4vmnlx457c1cstpgllx_ck_h0000gn/T/ipykernel_91724/628328432.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Outcome"

In [96]:
df.drop(['Notes', 'Name', 'Education', 'Company', 'Yes/No/Maybe'], axis=1, inplace=True)

In [114]:
df.head()

,Headline,Experiences,schools,national_rank,quality_of_education,number_founders,self_employed,Outcome,tokens,tokens_headline,tokens_experiences
0,Health & Wealth Strategist at WellBeing & Worth,"Founder @ WellbeingandWorth, Oct 2019 – Presen...",[Johns Hopkins University],13.0,18.0,2,False,0.0,"(Health, &, Wealth, Strategist, at, WellBeing,...","(Health, &, Wealth, Strategist, at, WellBeing,...","(Founder, @, WellbeingandWorth, ,, Oct, 2019, ..."
1,Founder at Aromatherapy University,Founder @ Aromatherapy University\n Self-empl...,"[University of California, Riverside, Johns Ho...",26.25,68.25,1,True,0.0,"(Founder, at, Aromatherapy, University)","(Founder, at, Aromatherapy, University)","(Founder, @, Aromatherapy, University, \n , S..."
2,Financial Advisor for Entrepreneurs and Tech P...,Founder @ Morton Financial Advice\n Self-empl...,[Johns Hopkins University],13.0,18.0,3,True,0.0,"(Financial, Advisor, for, Entrepreneurs, and, ...","(Financial, Advisor, for, Entrepreneurs, and, ...","(Founder, @, Morton, Financial, Advice, \n , ..."
3,Founder & CEO at Orion Applied Science & Techn...,Founder & CEO @ Orion Applied Science & Techno...,[Johns Hopkins University],13.0,18.0,2,False,0.0,"(Founder, &, CEO, at, Orion, Applied, Science,...","(Founder, &, CEO, at, Orion, Applied, Science,...","(Founder, &, CEO, @, Orion, Applied, Science, ..."
4,"Women's Wellness Exchange, Adventures Within, LLC","Founder @ Women's Wellness Exchange, Dec 2020 ...",[Johns Hopkins University],13.0,18.0,3,True,0.0,"(Women, 's, Wellness, Exchange, ,, Adventures,...","(Women, 's, Wellness, Exchange, ,, Adventures,...","(Founder, @, Women, 's, Wellness, Exchange, ,,..."


In [121]:
df['tokens_headline'] = [text.split() for text in df['Headline']]

In [122]:
df.head()

,Headline,Experiences,schools,national_rank,quality_of_education,number_founders,self_employed,Outcome,tokens,tokens_headline,tokens_experiences
0,Health & Wealth Strategist at WellBeing & Worth,"Founder @ WellbeingandWorth, Oct 2019 – Presen...",[Johns Hopkins University],13.0,18.0,2,False,0.0,"(Health, &, Wealth, Strategist, at, WellBeing,...","[Health, &, Wealth, Strategist, at, WellBeing,...","(Founder, @, WellbeingandWorth, ,, Oct, 2019, ..."
1,Founder at Aromatherapy University,Founder @ Aromatherapy University\n Self-empl...,"[University of California, Riverside, Johns Ho...",26.25,68.25,1,True,0.0,"(Founder, at, Aromatherapy, University)","[Founder, at, Aromatherapy, University]","(Founder, @, Aromatherapy, University, \n , S..."
2,Financial Advisor for Entrepreneurs and Tech P...,Founder @ Morton Financial Advice\n Self-empl...,[Johns Hopkins University],13.0,18.0,3,True,0.0,"(Financial, Advisor, for, Entrepreneurs, and, ...","[Financial, Advisor, for, Entrepreneurs, and, ...","(Founder, @, Morton, Financial, Advice, \n , ..."
3,Founder & CEO at Orion Applied Science & Techn...,Founder & CEO @ Orion Applied Science & Techno...,[Johns Hopkins University],13.0,18.0,2,False,0.0,"(Founder, &, CEO, at, Orion, Applied, Science,...","[Founder, &, CEO, at, Orion, Applied, Science,...","(Founder, &, CEO, @, Orion, Applied, Science, ..."
4,"Women's Wellness Exchange, Adventures Within, LLC","Founder @ Women's Wellness Exchange, Dec 2020 ...",[Johns Hopkins University],13.0,18.0,3,True,0.0,"(Women, 's, Wellness, Exchange, ,, Adventures,...","[Women's, Wellness, Exchange,, Adventures, Wit...","(Founder, @, Women, 's, Wellness, Exchange, ,,..."


In [128]:
from gensim import corpora
from numpy import linalg as LA

In [127]:
d = corpora.Dictionary(df['tokens_headline'])
corp = [d.doc2bow(line) for line in df['tokens_headline']]

In [129]:
corp_norm = [LA.norm(vec) for vec in corp]
df['tokens_headline'] = corp_norm

In [147]:
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [155]:
mlpreg = MLPClassifier(hidden_layer_sizes=(10,), activation='relu',
         solver="sgd", alpha=0.001, batch_size="auto",
         learning_rate="adaptive", learning_rate_init=0.01,
         power_t=0.5, max_iter=1000, shuffle=True, random_state=9,
         tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
         nesterovs_momentum=True, early_stopping=False,
         validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
         epsilon=1e-08)

In [162]:
X_train, X_test, y_train, y_test = train_test_split(df['tokens_headline'], df['Outcome'], random_state=1, stratify=df['Outcome'])

In [164]:
mlpreg.fit(pd.DataFrame(X_train), y_train)
mlpreg.score(pd.DataFrame(X_test), y_test)

0.9298780487804879

In [166]:
mlpreg.predict_proba(pd.DataFrame(X_test))

array([[0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.92485257, 0.04216262, 0.03298481],
       [0.

In [159]:
X_train

382     7853.693908
995     2977.000168
1037    2977.000168
573     2335.334023
471     5417.170479
           ...     
715     7841.324378
905     2977.000168
1096    2977.000168
235     2915.943072
1061    2977.000168
Name: tokens_headline, Length: 983, dtype: float64